In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from functools import lru_cache
from pathlib import Path
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
import cv2
import datasets
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import torch
from PIL import Image
from torch.utils.data import DataLoader
import nnx.data
from nnx.data.ctspine1k.dataset import CTSpine1K, SAMAdapter
from nnx.data.data_structures import AnnotationSample, ImagePrompt, SAMSample
from nnx.data.plots import visualize_sam_sample
from torch import nn
from tqdm.auto import tqdm
from training.loss_fns import iou_loss, sigmoid_focal_loss

nnx.data.rng = np.random.default_rng(seed=1)

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
SAM2_CHECKPOINT = "../checkpoints/sam2.1_hiera_large.pt"
MODEL_CONFIG = "configs/sam2.1/sam2.1_hiera_l.yaml"

/Users/alexanderdann/Documents/Privat/Code/Repositories/computer-vision-projects/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cache_dir = Path("/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K")
files = [file.parent for file in cache_dir.rglob("*") if file.suffix == ".gz"]
files

[PosixPath('/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K/0001_CT/1/NIFTI'),
 PosixPath('/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K/0001_CT/1/NIFTI'),
 PosixPath('/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K/0005_CT/1/NIFTI'),
 PosixPath('/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K/0005_CT/1/NIFTI'),
 PosixPath('/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K/0004_CT/1/NIFTI'),
 PosixPath('/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K/0004_CT/1/NIFTI'),
 PosixPath('/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K/0003_CT/1/NIFTI'),
 PosixPath('/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K/0003_CT/1/NIFTI'),
 PosixPath('/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K/0002_CT/1/NIFTI'),
 PosixPath('/Users/alexanderdann/Documents/Privat/Code/Data/CTSpine1K/0002_CT/1/NIFTI')]

In [4]:
sam2_model = build_sam2(MODEL_CONFIG, SAM2_CHECKPOINT, DEVICE)

predictor = SAM2ImagePredictor(sam2_model)

#predictor.model.sam_mask_decoder.train(mode=True)
#predictor.model.sam_prompt_encoder.train(mode=True)

In [5]:
ct_dataset = CTSpine1K(cache_dir=cache_dir)
dataset = SAMAdapter(dataset=ct_dataset, image_size=predictor.model.image_size, use_torch=True)


Found 2825 samples.


In [ ]:
def to_device(data, device):
    """Recursively move data to device."""
    if isinstance(data, torch.Tensor):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [to_device(item, device) for item in data]
    elif isinstance(data, dict):
        return {k: to_device(v, device) for k, v in data.items()}
    elif isinstance(data, torch.nested.NestedTensor):
        return data.to(device)
    else:
        # If it's not a tensor or container of tensors, return as is
        return data

In [ ]:
dataloader = DataLoader(dataset, collate_fn=dataset.collate_fn, num_workers=16)

In [7]:
sample_ = None
for sample in tqdm(dataloader):
    sample_ = sample

  0%|          | 0/2825 [00:00<?, ?it/s]/Users/alexanderdann/Documents/Privat/Code/Repositories/computer-vision-projects/venv/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/NestedTensorImpl.cpp:179.)
  return _nested.nested_tensor(
/Users/alexanderdann/Documents/Privat/Code/Repositories/computer-vision-projects/venv/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/NestedTensorImpl.cpp:179.)
  return _nested.nested_tensor(
/Users/alexanderdann/Documents/Privat/Code/Repositories/computer-vision-projects/venv/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of n

KeyboardInterrupt: 

In [ ]:
sample_["point_coords"]

[[tensor([[[142., 282.],
           [ 97., 305.],
           [144., 317.],
           [143., 314.],
           [157., 330.]]]),
  tensor([[[170., 336.],
           [158., 331.],
           [213., 236.]]])]]

In [ ]:
sample = dataset[285]
sample

In [ ]:
sample[0].shape

In [ ]:
torch.nested.nested_tensor()

In [ ]:
predictor.set_image(sample[0])

sparse_embeddings, dense_embeddings = predictor.model.sam_prompt_encoder(
    points=sample[2][0],
    boxes=None,
    masks=None,
)

low_res_masks, prediction_scores, sam_tokens_out, object_score_logits = (
    predictor.model.sam_mask_decoder(
        image_embeddings=predictor.get_image_embedding(),
        image_pe=predictor.model.sam_prompt_encoder.get_dense_pe(),
        sparse_prompt_embeddings=sparse_embeddings,
        dense_prompt_embeddings=dense_embeddings,
        multimask_output=False,
        repeat_image=False,
        high_res_features=predictor.get_high_resolution_features(),
    )
)
pred_masks = predictor.mask_postprocessing(low_res_masks)

In [ ]:
pred_masks.shape

In [ ]:
import sam2

In [ ]:
iou_loss(pred_masks, sample[1][0][None][None], prediction_scores,pred_masks.size(1) )
sigmoid_focal_loss(pred_masks, sample[1][0][None][None],pred_masks.size(1) )

In [ ]:
sigmoid_focal_loss(pred_masks, sample[1][0][None][None])

In [ ]:
iou_loss(pred_masks, sample[1][0][None][None])

cross_entropy = nn.BCEWithLogitsLoss(weight=torch.Tensor([[1, 10]]))
num_batches, num_classes = pred_masks.shape[:2]

cross_entropy(pred_masks.reshape(num_batches, num_classes, -1), sample[1][0][None][None].reshape(num_batches, num_classes, -1))

In [ ]:
def find_closests(y_)

In [ ]:
masks = pred_masks[0][0].sigmoid().detach().numpy() > 0.5

plt.imshow((masks).astype(np.float32))

In [ ]:
plt.imshow(sample[1][0].numpy())